In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [20]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [22]:
def preprocess_our_df(df):
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['FareBin'] = pd.qcut(df['Fare'], 4, labels = ['Low', 'Medium', 'High', 'Very High'])
    
    return df

In [23]:
train = preprocess_our_df(train)
test = preprocess_our_df(test)

In [24]:
# selecting features based on importance figured out prev
features = ['Pclass', 'Sex', 'Fare', 'Title', 'IsAlone', 'FareBin']

X = train[features]
y = train['Survived']

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [52]:
numeric_features = ['Fare']
categorical_features = ['Pclass', 'Sex', 'Title', 'IsAlone', 'FareBin']

preprocessor = ColumnTransformer(
    transformers = [
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy = 'median')),
            ('scaler', StandardScaler())]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
            ('onehot', OneHotEncoder(handle_unknown = 'ignore'))]), categorical_features)])

In [53]:
pipeline = Pipeline([('preprocessor', preprocessor)])

In [54]:
X_train_processed = pipeline.fit_transform(X_train)
X_val_processed = pipeline.transform(X_val)

In [60]:
def creating_model(input_dim):
    model = Sequential([
        Dense(64, activation = 'relu', kernel_regularizer = l2(0.01), input_shape = (input_dim,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation = 'relu', kernel_regularizer = l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(16, activation = 'relu', kernel_regularizer = l2(0.01)),
        BatchNormalization(),
        Dense(1, activation = 'sigmoid')])
    
    model.compile(optimizer = Adam(learning_rate = 0.001),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [61]:
model = creating_model(X_train_processed.shape[1])

In [63]:
history = model.fit(
    X_train_processed, y_train,
    validation_data = (X_val_processed, y_val),
    epochs = 200,
    batch_size = 32)
    # , verbose = 0)

Epoch 1/200
23/23 [==============================] - 4s 40ms/step - loss: 1.6741 - accuracy: 0.5843 - val_loss: 1.5480 - val_accuracy: 0.5922
Epoch 2/200
23/23 [==============================] - 0s 13ms/step - loss: 1.5088 - accuracy: 0.6784 - val_loss: 1.4978 - val_accuracy: 0.5866
Epoch 3/200
23/23 [==============================] - 0s 11ms/step - loss: 1.3794 - accuracy: 0.7683 - val_loss: 1.4594 - val_accuracy: 0.5978
Epoch 4/200
23/23 [==============================] - 0s 14ms/step - loss: 1.3426 - accuracy: 0.7753 - val_loss: 1.4238 - val_accuracy: 0.5866
Epoch 5/200
23/23 [==============================] - 0s 13ms/step - loss: 1.2714 - accuracy: 0.7837 - val_loss: 1.3915 - val_accuracy: 0.5866
Epoch 6/200
23/23 [==============================] - 0s 14ms/step - loss: 1.2328 - accuracy: 0.7893 - val_loss: 1.3562 - val_accuracy: 0.5922
Epoch 7/200
23/23 [==============================] - 0s 13ms/step - loss: 1.1930 - accuracy: 0.8076 - val_loss: 1.3215 - val_accuracy: 0.5922
Epoch 

In [64]:
train_accuracy = model.evaluate(X_train_processed, y_train, verbose = 0)[1]
val_accuracy = model.evaluate(X_val_processed, y_val, verbose = 0)[1]

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

Train Accuracy: 0.8455
Validation Accuracy: 0.8045


In [65]:
X_test_processed = pipeline.transform(test[features])
predictions = model.predict(X_test_processed)
predictions = (predictions > 0.5).astype(int).flatten()

14/14 [==============================] - 1s 7ms/step


In [66]:
# submission try #
i = 4

In [67]:
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': predictions})
submission.to_csv(f'submission{i}.csv', index= False)

print("Submission file created.")

Submission file created.


In [68]:
# submission4.csv is the last one so far, but we're out of submission attempts
# last submission file was submission3.csv

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0a5b4bfd-ca80-4e47-8996-e46ecfe92190' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>